In [1]:
!pip install konlpy
!pip install pandas openpyxl xlrd konlpy scikit-learn

!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 26.8 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 3s (4,084 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to 

In [2]:
import pandas as pd
import numpy as np
from konlpy.tag import Komoran
import re

komoran = Komoran()
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from konlpy.tag import Okt
okt = Okt()

In [3]:
def tokenize_and_lemmatize(text):
    if isinstance(text, str):
        # 사전 전처리: 콜론과 쉼표를 제거하거나, 대체 문자열로 변경
        text = text.replace(':', ' ').replace(',', ' ')

        # 토큰화 및 어간 추출
        tokens = okt.morphs(text, stem=True)

        return ' '.join(tokens)
    else:
        return ''

In [4]:
pikle_data=pd.read_excel('/content/moviesdata0808.xls', header=None)
column_names=['영화명', '평점', '연도', '상영시간', '연령', '감독', '출연', '줄거리', '장르']
pikle_data.columns = column_names
pikle_data.drop(index=0, inplace=True)
df=pd.DataFrame(pikle_data, columns=['영화명', '평점', '연도', '상영시간', '연령', '감독', '출연', '줄거리', '장르'])
data=df
data.reset_index(drop=True, inplace=True)
data.to_csv('movie_data')
data['Text'] = (data['장르'] + ' ' + data['연령']  + ' ' + data['감독'] + ' ' + data['출연']+ ' ' + data['줄거리'])
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from konlpy.tag import Okt


# 텍스트 데이터에 함수 적용
data['Processed_Text'] = data['Text'].apply(tokenize_and_lemmatize)
data['synopsis_processed'] = data['줄거리'].apply(tokenize_and_lemmatize)
# 처리된 텍스트 데이터 벡터화
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(data['Processed_Text'])
tfidf_matrix_synopsys = tfidf.transform(data['synopsis_processed'])
# 코사인 유사도 행렬 계산
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
# 주어진 리스트
original_list = [
    "SF", "액션", "서부극", "미스터리", "전쟁", "스릴러", "코미디",
    "애니메이션", "범죄", "가족", "뮤지컬", "드라마", "기타", "다큐멘터리",
    "어드벤처", "사극", "공연", "공포(호러)", "멜로/로맨스", "판타지"
]

# 기준이 되는 리스트
desired_order = [
    "액션/서부극", "드라마", "SF/무협/판타지/어드벤처", "범죄/수사/추리/스릴러",
    "로맨틱코미디", "멜로/로맨스", "코미디", "애니메이션", "사극/시대극", "공포",
    "다큐멘터리", "성인/에로", "실험/예술영화", "기타"
]

# 카테고리 매핑
category_mapping = {
    "액션": "액션/서부극(웨스턴)",
    "서부극": "액션/서부극",
    "SF": "SF/무협/판타지/어드벤처",
    "판타지": "SF/무협/판타지/어드벤처",
    "무협": "SF/무협/판타지/어드벤처",
    "어드벤처": "SF/무협/판타지/어드벤처",
    "범죄": "범죄/수사/추리/스릴러",
    "수사": "범죄/수사/추리/스릴러",
    "추리": "범죄/수사/추리/스릴러",
    "스릴러": "범죄/수사/추리/스릴러",
    "로맨틱코미디": "로맨틱코미디",
    "멜로/로맨스": "멜로/로맨스",
    "코미디": "코미디",
    "애니메이션": "애니메이션",
    "사극": "사극/시대극",
    "시대극": "사극/시대극",
    "공포(호러)": "공포",
    "다큐멘터리": "다큐멘터리",
    "성인/에로": "성인/에로",
    "실험/예술영화": "실험/예술영화",
    "기타": "기타",
    "드라마": "드라마",
    "미스터리": "범죄/수사/추리/스릴러",
    "전쟁": "기타",
    "가족": "기타",
    "뮤지컬": "기타",
    "공연": "기타"
}

# 항목 분류 및 정렬
classified = []
unclassified = []
categorys = []
for item in original_list:
    category = category_mapping.get(item, None)
    categorys.append(category)
    if category:
        classified.append(category)
    else:
        unclassified.append(item)

# 중복 제거하고 정렬
classified = list(dict.fromkeys(classified))

# 정렬된 리스트 생성
sorted_list = [category for category in desired_order if category in classified]
sorted_list.extend(unclassified)

print(sorted_list)
genre_favor=pd.read_excel('/content/연령별 장르 선호도.xlsx')
genre_favor.drop(index=0, inplace=True)
genre_favor.columns
genre_favor.drop(columns=[' '], inplace=True)
genre_favor.columns

genre_favor['로맨틱코미디/멜로/로맨스']=genre_favor['로맨틱코미디']+genre_favor['멜로/로맨스']
genre_favor['기타']=genre_favor['기타']+genre_favor['실험/예술영화']+genre_favor['성인/에로']
genre_favor.drop(columns=['로맨틱코미디', '멜로/로맨스', '실험/예술영화', '성인/에로'], inplace=True)
genre_favor.drop(columns='사례수',inplace=True)
new_order=['Unnamed: 1', '액션', 'SF/판타지/무협', '드라마', '범죄/수사/추리/스릴러', '로맨틱코미디/멜로/로맨스', '사극/시대극', '코미디',
       '애니메이션', '다큐멘터리', '공포', '기타', '합계']
genre_favor=genre_favor[new_order]
new_columns=['연령대','액션/서부극(웨스턴)','SF/무협/판타지/어드벤처', '드라마', '범죄/수사/추리/스릴러', '로맨틱코미디/멜로/로맨스','사극/시대극', '코미디', '애니메이션',
             '다큐멘터리', '공포', '기타','합계']
genre_favor.columns=new_columns
genre_favor.loc[1,'연령대']= '0~18세 남성'
genre_favor.loc[7,'연령대']= '50~90세 남성'
genre_favor.loc[8,'연령대']= '0~18세 여성'
genre_favor.loc[14,'연령대']= '50~90세 여성'
genre_favor_divided=genre_favor.iloc[:,1:13]/100
genre_favor_divided.insert(0,'연령대',genre_favor['연령대'])
genre_weights_df=genre_favor_divided
genre_weights_df

['액션/서부극', '드라마', 'SF/무협/판타지/어드벤처', '범죄/수사/추리/스릴러', '멜로/로맨스', '코미디', '애니메이션', '사극/시대극', '공포', '다큐멘터리', '기타']


,연령대,액션/서부극(웨스턴),SF/무협/판타지/어드벤처,드라마,범죄/수사/추리/스릴러,로맨틱코미디/멜로/로맨스,사극/시대극,코미디,애니메이션,다큐멘터리,공포,기타,합계
1,0~18세 남성,0.393939,0.287879,0.060606,0.075758,0.045455,0.015152,0.060606,0.015152,0.0,0.015152,0.030303,1.0
2,19~23세 남성,0.370079,0.228346,0.102362,0.102362,0.110236,0.007874,0.031496,0.007874,0.0,0.031496,0.007874,1.0
3,24~29세 남성,0.33125,0.21875,0.1625,0.0875,0.1125,0.025,0.04375,0.00625,0.0,0.0,0.0125,1.0
4,30~34세 남성,0.373016,0.166667,0.15873,0.039683,0.111111,0.071429,0.047619,0.007937,0.007937,0.007937,0.007937,1.0
5,35~39세 남성,0.471698,0.198113,0.075472,0.018868,0.103774,0.028302,0.028302,0.028302,0.018868,0.0,0.028302,1.0
6,40~49세 남성,0.421801,0.222749,0.085308,0.033175,0.099526,0.052133,0.037915,0.023697,0.0,0.004739,0.018957,1.0
7,50~90세 남성,0.361111,0.127778,0.111111,0.061111,0.094444,0.133333,0.022222,0.005556,0.061111,0.0,0.022222,1.0
8,0~18세 여성,0.0625,0.15625,0.1875,0.171875,0.265625,0.046875,0.015625,0.0625,0.0,0.015625,0.015625,1.0
9,19~23세 여성,0.175,0.208333,0.125,0.075,0.316667,0.008333,0.041667,0.008333,0.0,0.025,0.016667,1.0
10,24~29세 여성,0.23871,0.129032,0.109677,0.135484,0.296774,0.032258,0.019355,0.012903,0.0,0.019355,0.006452,1.0


In [5]:
import pandas as pd

# 연령대와 성별에 따른 가중치 데이터프레임 생성
genre_weights_data = {
    '연령대': [
        '0~18세 남성', '19~23세 남성', '24~29세 남성', '30~34세 남성',
        '35~39세 남성', '40~49세 남성', '50~90세 남성',
        '0~18세 여성', '19~23세 여성', '24~29세 여성', '30~34세 여성',
        '35~39세 여성', '40~49세 여성', '50~90세 여성'
    ],
    '액션/서부극(웨스턴)': [
        0.393939, 0.370079, 0.33125, 0.373016, 0.471698, 0.421801, 0.361111,
        0.0625, 0.175, 0.23871, 0.191667, 0.096154, 0.221154, 0.148571
    ],
    'SF/무협/판타지/어드벤처': [
        0.287879, 0.228346, 0.21875, 0.166667, 0.198113, 0.222749, 0.127778,
        0.15625, 0.208333, 0.129032, 0.091667, 0.105769, 0.144231, 0.091429
    ],
    '드라마': [
        0.060606, 0.102362, 0.1625, 0.15873, 0.075472, 0.085308, 0.111111,
        0.1875, 0.125, 0.109677, 0.191667, 0.259615, 0.254808, 0.245714
    ],
    '범죄/수사/추리/스릴러': [
        0.075758, 0.102362, 0.0875, 0.039683, 0.018868, 0.033175, 0.061111,
        0.171875, 0.075, 0.135484, 0.108333, 0.067308, 0.081731, 0.045714
    ],
    '로맨틱코미디/멜로/로맨스': [
        0.045455, 0.110236, 0.1125, 0.111111, 0.103774, 0.099526, 0.094444,
        0.265625, 0.316667, 0.296774, 0.25, 0.201923, 0.163462, 0.28
    ],
    '사극/시대극': [
        0.015152, 0.007874, 0.025, 0.071429, 0.028302, 0.052133, 0.133333,
        0.046875, 0.008333, 0.032258, 0.033333, 0.028846, 0.048077, 0.068571
    ],
    '코미디': [
        0.060606, 0.031496, 0.04375, 0.047619, 0.028302, 0.037915, 0.022222,
        0.015625, 0.041667, 0.019355, 0.008333, 0.038462, 0.028846, 0.017143
    ],
    '애니메이션': [
        0.015152, 0.007874, 0.00625, 0.007937, 0.028302, 0.023697, 0.005556,
        0.0625, 0.008333, 0.012903, 0.1, 0.163462, 0.043269, 0.005714
    ],
    '다큐멘터리': [
        0.0, 0.0, 0.0, 0.007937, 0.0, 0.0, 0.061111,
        0.0, 0.0, 0.0, 0.0, 0.009615, 0.004808, 0.051429
    ],
    '공포': [
        0.015152, 0.031496, 0.0, 0.007937, 0.0, 0.004739, 0.0,
        0.015625, 0.025, 0.019355, 0.016667, 0.028846, 0.009615, 0.017143
    ],
    '기타': [
        0.030303, 0.007874, 0.0125, 0.007937, 0.028302, 0.018957, 0.022222,
        0.015625, 0.016667, 0.006452, 0.008333, 0.0, 0.0, 0.028571
    ],
    '합계': [
        1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0
    ]
}

# 데이터프레임 생성
genre_weights_df = pd.DataFrame(genre_weights_data)

# 연령대와 성별을 받아 해당하는 가중치 행을 찾는 함수
def get_genre_weights(age, gender):
    # 성별에 따른 문자열 결정


    # 주어진 나이에 따라 연령대 범위를 결정
    if age == 10:
        age_range = '0~18세'
    elif age == 20:
        age_range = '19~23세'
    elif age == 30:
        age_range = '30~34세'
    elif age == 40:
        age_range = '40~49세'
    elif age == 50:
        age_range = '50~90세'
    elif age >= 60:
        age_range = '50~90세'
    else:
        raise ValueError("유효하지 않은 나이 범위입니다.")

    age_gender_key = f"{age_range} {gender}"

    # 해당 연령대와 성별에 맞는 가중치 행 선택 및 평균 계산
    if age_gender_key == '19~23세 남성':
        weights = (genre_weights_df.loc[genre_weights_df['연령대'] == age_gender_key].reset_index(drop=True).iloc[:, 1:] +
                   genre_weights_df.loc[genre_weights_df['연령대'] == '24~29세 남성'].reset_index(drop=True).iloc[:, 1:]) / 2
    elif age_gender_key == '30~34세 남성':
        weights = (genre_weights_df.loc[genre_weights_df['연령대'] == age_gender_key].reset_index(drop=True).iloc[:, 1:] +
                   genre_weights_df.loc[genre_weights_df['연령대'] == '35~39세 남성'].reset_index(drop=True).iloc[:, 1:]) / 2
    elif age_gender_key == '19~23세 여성':
        weights = (genre_weights_df.loc[genre_weights_df['연령대'] == age_gender_key].reset_index(drop=True).iloc[:, 1:] +
                   genre_weights_df.loc[genre_weights_df['연령대'] == '24~29세 여성'].reset_index(drop=True).iloc[:, 1:]) / 2
    elif age_gender_key == '30~34세 여성':
        weights = (genre_weights_df.loc[genre_weights_df['연령대'] == age_gender_key].reset_index(drop=True).iloc[:, 1:] +
                   genre_weights_df.loc[genre_weights_df['연령대'] == '35~39세 여성'].reset_index(drop=True).iloc[:, 1:]) / 2
    else:
        weights = genre_weights_df.loc[genre_weights_df['연령대'] == age_gender_key].iloc[:, 1:]

    return weights

genre_weights_df.to_csv('genre_geder_favor_df.csv')

In [6]:
movie_name_genre= pd.read_pickle('/content/movie_genre13306.pkl')
movie_name_genre=pd.DataFrame(movie_name_genre)

In [7]:

# CSV 파일을 Pandas DataFrame으로 불러오기
file_path = '/content/movie_data'
data = pd.read_csv(file_path)

# 'Unnamed: 0' 열 제거
data = data.drop(columns=['Unnamed: 0'])

# 열 이름을 한국어로 설정
data.columns = ['영화명', '평점', '연도', '상영시간', '연령', '감독', '출연진', '줄거리', '장르']

pd.set_option('display.max_columns', None)

# 한국어 불용어 정의
stop_words_ko = {"및", "도", "에", "의", "가", "이", "은", "는", "을", "를", "에서", "로", "과", "와", "한", "그", "이", "하", "여", "속", "전", "자", "이다"}

# 장르 동의어 매핑 생성
genre_synonyms_ko = {
    '공포': ['공포', '호러', '스릴러', '무서운'],
    '로맨스': ['로맨스', '사랑', '연애', '로맨틱'],
    '멜로': ['멜로', '멜로드라마'],
    '액션': ['액션', '모험', '전투', '스릴'],
    '코미디': ['코미디', '웃긴', '유머', '희극'],
    '서부': ['서부', '웨스턴', '카우보이', '서부극', 'western'],
}

In [8]:
# 장르 동의어를 정규화하는 함수
def normalize_genre_from_synonyms_ko(keyword):
    keyword = keyword.strip().lower()
    for genre, synonyms in genre_synonyms_ko.items():
        if keyword in [synonym.lower() for synonym in synonyms]:
            return genre
    return keyword

# 장르 정규화 함수 (괄호 및 슬래시 처리 포함)
def normalize_genre(text):
    if pd.isna(text):
        return ''
    text = re.sub(r'\s*\(/?\s*|\s*/\s*|\s*\)\s*', ',', text)
    text = re.sub(r'\s+', ' ', text)
    genres = [normalize_genre_from_synonyms_ko(genre.strip()) for genre in text.split(',')]
    return ','.join(sorted(set(genres)))


def preprocessing(user_input, komoran, remove_stopwords=False, stop_words=[]):
    # 한글, 숫자, S/F 문자만 남김
    review_text = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ0-9\\sSFsf]", "", user_input)

    # 형태소 분석
    word_review = komoran.pos(review_text, flatten=False)

    # 고유명사(NNP)와 일반명사(NNG)를 분리
    filtered_review = []      # 고유명사(NNP)만 담을 리스트
    nnp_and_nng_review = []   # 고유명사(NNP)와 일반명사(NNG) 모두 담을 리스트

    for sentence in word_review:  # 문장 단위로 분리된 리스트
        for word, tag in sentence:
            if tag == 'NNP':
                filtered_review.append((word, tag))  # 고유명사만 담기
                nnp_and_nng_review.append((word, tag))  # 고유명사도 같이 담기
            elif tag == 'NNG':
                nnp_and_nng_review.append((word, tag))  # 일반명사 담기

    if remove_stopwords:
        filtered_review = [token for token in filtered_review if not token[0] in stop_words]
        nnp_and_nng_review = [token for token in nnp_and_nng_review if not token[0] in stop_words]

    return filtered_review, nnp_and_nng_review

def find_matching_movie(filtered_review, movie_name_genre):
    # filtered_review가 비어 있지 않은 경우에만 수행
    if filtered_review:
        for word, tag in filtered_review:
            for movie_name in movie_name_genre['영화명']:
                if re.search(word, movie_name):
                    return movie_name_genre[movie_name_genre['영화명'] == movie_name]

    return None

def find_matching_cast(filtered_review, cast_members):
    # filtered_review가 비어 있지 않은 경우에만 수행
    if filtered_review:
        matching_cast = []
        for word, tag in filtered_review:
            if word in cast_members:
                matching_cast.append(word)

        return matching_cast[0] if matching_cast else None

    return None

def find_matching_director(filtered_review, directors):
    # filtered_review가 비어 있지 않은 경우에만 수행
    if filtered_review:
        matching_director = []
        for word, tag in filtered_review:
            if word in directors:
                matching_director.append(word)

        return matching_director[0] if matching_director else None

    return None

def find_matching_genre(nnp_and_nng_review, genres):
    # nnp_and_nng_review가 비어 있지 않은 경우에만 수행
    if nnp_and_nng_review:
        matching_genres = []
        genre_list = genres  # 장르 목록을 리스트로 변환하여 일치 여부를 빠르게 검사

        for word, tag in nnp_and_nng_review:
            # 명사 사이에 띄어쓰기가 있을 경우, 분리하여 장르를 찾기
            words_to_check = word.split()  # 띄어쓰기로 분리된 단어들

            for w in words_to_check:
                if w in genre_list:
                    matching_genres.append(w)

        # 중복된 장르를 제거하기 위해 set을 사용한 후 리스트로 변환
        matching_genres = list(set(matching_genres))
        return ', '.join(matching_genres)

    return None


In [9]:
# 영화 정보 딕셔너리 생성
movie_info = {}
for i, row in data.iterrows():
    movie_id = i
    content = row.to_dict()
    movie_info[movie_id] = content

# 영화 제목과 정보를 매핑하는 새로운 DataFrame 생성
movie_titles = data[['영화명']].rename(columns={'영화명': '제목'})
movie_titles['info'] = movie_titles.index.map(lambda x: movie_info[x])

# DataFrame의 장르 정규화
movie_titles['info'] = movie_titles['info'].apply(lambda x: {**x, '장르': normalize_genre(x['장르'])})

In [10]:
# 장르로 영화 추천
def recommend_movies_by_genre(genre):
    normalized_genre = normalize_genre_from_synonyms_ko(genre)
    recommended_movies = movie_titles[movie_titles['info'].apply(lambda x: '장르' in x and normalized_genre in x['장르'])]
    return recommended_movies[['제목', 'info']]

# 감독 또는 출연 배우로 영화 추천
def recommend_movies_by_person(person_name):
    person_name = person_name.strip().lower()
    recommended_movies = movie_titles[movie_titles['info'].apply(
        lambda x: ('감독' in x and person_name in x['감독'].strip().lower())
    ) | movie_titles['info'].apply(
        lambda x: ('출연진' in x and person_name in x['출연진'].strip().lower())
    )]
    return recommended_movies[['제목', 'info']]

# 장르 목록을 가져오는 함수
def get_genres():
    genres = set()
    for info in movie_titles['info']:
        if '장르' in info:
            text = info['장르']
            genres.update([normalize_genre_from_synonyms_ko(genre.strip()) for genre in text.split(',')])
    return sorted(genres)

# 영화 제목으로 정보를 가져오는 함수
def get_movie_info(title):
    filtered_title = title.strip()
    matching_titles = movie_titles[movie_titles['제목'].str.contains(filtered_title, na=False)]

    if matching_titles.empty:
        return f"영화를 찾지 못했습니다: {title}"

    movie_id = matching_titles.index[0]
    return movie_titles[movie_titles.index == movie_id]['info'].iloc[0]

In [11]:
# 영화명, 감독, 출연 배우, 장르 및 인물 찾기
genres = get_genres()
# movies = data['영화명'].dropna().apply(lambda x: x.split(', ')).explode().unique()
directors = data['감독'].dropna().apply(lambda x: x.split(', ')).explode().unique()
cast_members = data['출연진'].dropna().apply(lambda x: x.split(', ')).explode().unique()
print(f"데이터에 있는 감독들: {directors}")
print(f"데이터에 있는 출연 배우들: {cast_members}")

데이터에 있는 감독들: ['토드 필드' '데이비드 지달리' '피터 버그' ... '핀 테일러' '양익준' '킬리언 리드호퍼']
데이터에 있는 출연 배우들: ['케이트 블란쳇' '노에미 메를랑' '니나 호스' ... '파노우' '데닉' '야니스 니에브외너']


In [12]:
movie_name_genre['text']= movie_name_genre['영화명'] +' '+ movie_name_genre['장르']

In [51]:
def get_recommendations(movie_title=None,keywords=None, genre=None, director=None, actor = None, gender=None, age=None, favored_genres=None, cosine_sim=cosine_sim):
    print(f"함수 내부에서 받은 장르: {genre}")  # 장르 값 확인
    filtered_data = data.copy()

    # 기본 가중치 설정
    weights = {'genre_weight': 0.0, 'director_weight': 0.0}
    entered_criteria = 0

    # 입력된 조건에 대해 가중치 증가
    if genre:
        weights['genre_weight'] = 0.2
        entered_criteria += 1
    if director:
        weights['director_weight'] = 0.2
        entered_criteria += 1
    if actor:
        weights['actor_weight'] = 0.2
        entered_criteria += 1


    #가입 할 때 기입하는 선호 장르 가중치
    if favored_genres:
        weights['favored_genre_weight'] = 0.1

    # 연령과 성별에 따른 장르 가중치 계산
    if gender and age:
        genre_weights = get_genre_weights(age, gender)  # 여성일 때 드라마 장르 가중치
        print(genre_weights)
    print(genre)

    if movie_title:
        if movie_title not in data['영화명'].values:
            # 코사인 유사도 계산 없이 가장 유사한 영화명 추천
            # print(movie_title)
            movie_text = movie_name_genre[movie_name_genre['영화명'] == movie_title]['text'].values
            if len(movie_text) > 0:
                # 해당 텍스트를 토큰화 및 전처리 후 벡터화
                processed_text = tokenize_and_lemmatize(movie_text[0])
                tfidf_title_genre = tfidf.transform([processed_text])

                # 코사인 유사도 계산
                cosine_sim_title = linear_kernel(tfidf_title_genre, tfidf_matrix)
                sim_scores = list(enumerate(cosine_sim_title[0]))
                print(sim_scores)
            # 입력조건, 연령과 성별에 따른 장르 가중치 적용
            for i, (idx, score) in enumerate(sim_scores):
                #입력 조건 가중치 적용
                #장르 가중치
                genre_score = 0
                if genre:
                    # 입력된 장르를 쉼표로 구분하여 소문자로 변환하고 공백을 제거한 후 리스트로 만듦
                    input_genres = [g.strip().lower() for g in genre.split(',')]
                    # 영화의 장르를 소문자로 변환하고 공백을 제거한 후 리스트로 만듦
                    searched_m_genres = [g.strip().lower() for g in data.loc[idx, '장르'].split(',')]
                    # 입력된 각 장르에 대해 영화의 장르 목록과 비교하여 일치하는 경우 가중치 적용
                    genre_score = 0  # 장르 점수 초기화
                    for input_genre in input_genres:
                        if input_genre in searched_m_genres:
                            genre_score += weights['genre_weight']  # 일치하는 장르마다 가중치 추가

                #감독, 배우 가중치
                director_score = weights['director_weight'] if director and director in data.loc[idx, '감독'] else 0
                actor_score = weights['actor_weight'] if actor and actor in data.loc[idx, '출연진'] else 0

                #해당 영화의 장르 확인
                genre_list = data.loc[idx, '장르'].split(',')

                # 초기화
                genre_weight_score = 0
                # 장르 가중치 리스트 초기화
                genre_weights_list = []

                # 장르 가중치 계산
                if gender and age:
                    for genre1 in genre_list:
                        genre1 = genre1.strip()
                        for column in genre_weights.columns:
                            if any(genre_part.strip() == genre1 for genre_part in column.split('/')):
                                genre_weight = genre_weights[column].values[0]
                                genre_weights_list.append(genre_weight)  # 각 장르 가중치를 리스트에 추가
                                break

                    # 가장 큰 두 개의 가중치를 선택하여 합산
                    if len(genre_weights_list) >= 2:
                        genre_weights_list.sort(reverse=True)
                        genre_weight_score = genre_weights_list[0] + genre_weights_list[1]
                        genre_weight_score = genre_weight_score/2  # 두 개의 가중치를 반영한 점수 계산
                    else:
                        genre_weight_score = sum(genre_weights_list)  # 장르가 두 개 미만일 경우 모든 가중치를 합산

                # 선호 장르 가중치 적용
                favored_genre_weight_score = 0
                if favored_genres:
                    for favor_genre in favored_genres:
                        if favor_genre in genre_list:
                            favored_genre_weight_score += weights['favored_genre_weight']

                # 기존 점수와 새로운 가중치를 반영한 점수 계산
                total_score = score + genre_score + director_score + actor_score +  genre_weight_score + favored_genre_weight_score
                # 디버깅 또는 로그 출력을 위해 idx와 관련 정보를 출력
                # print(f'Idx: {idx}, Movie Title: {data.loc[idx, "영화명"]}, '
                #     f'Similarity: {score}, Genre Score: {genre_score}, '
                #     f'Director Score: {director_score},'
                #     f'Genre Weight Score: {genre_weight_score},'
                #     f'favored_genre_weight_score: {favored_genre_weight_score},'
                #     f'Total Score: {total_score}' )

                # 최종 점수와 함께 저장
                sim_scores[i] = (idx, total_score)
            print(movie_title,genre,director,actor)
            # 유사도 순으로 정렬
            sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
            print(sim_scores)
            sim_scores = sim_scores[:5]
            movie_indices = [i[0] for i in sim_scores]
            recommendations = data.iloc[movie_indices]

            # 시각화를 위한 유사도 점수 포함
            similarity_scores = [i[1] for i in sim_scores]
            return recommendations[['영화명', '평점', '연도', '상영시간', '연령', '감독', '출연진', '줄거리', '장르']].values.tolist(), similarity_scores
        else: #영화 제목이 데이터에 있는 경우
            # 영화명에 해당하는 인덱스 가져오기
            idx = data[data['영화명'] == movie_title].index[0]

            # 필터링된 데이터 내에서 영화명과 유사한 영화들을 추천하기 위해 코사인 유사도 계산
            filtered_indices = filtered_data.index

            sim_scores = []
            for filtered_idx in filtered_indices:
                if filtered_idx == idx:
                    continue  # 자기 자신은 제외
                sim_score = cosine_sim[idx, filtered_idx]


                # 가중치 적용
                #장르 가중치
                genre_score = 0
                if genre:
                    # 입력된 장르를 쉼표로 구분하여 소문자로 변환하고 공백을 제거한 후 리스트로 만듦
                    input_genres = [g.strip().lower() for g in genre.split(',')]
                    # 영화의 장르를 소문자로 변환하고 공백을 제거한 후 리스트로 만듦
                    searched_m_genres = [g.strip().lower() for g in data.loc[filtered_idx, '장르'].split(',')]
                    # 입력된 각 장르에 대해 영화의 장르 목록과 비교하여 일치하는 경우 가중치 적용
                    genre_score = 0  # 장르 점수 초기화
                    for input_genre in input_genres:
                        if input_genre in searched_m_genres:
                            genre_score += weights['genre_weight']  # 일치하는 장르마다 가중치 추가

                #감독, 배우 가중치
                director_score = weights['director_weight'] if director and director in data.loc[filtered_idx, '감독'] else 0
                actor_score = weights['actor_weight'] if actor and actor in data.loc[filtered_idx, '출연진'] else 0


                #해당 영화의 장르 확인
                genre_list = data.loc[filtered_idx, '장르'].split(',')


                # 초기화
                genre_weight_score = 0
                # 장르 가중치 리스트 초기화
                genre_weights_list = []
                if gender and age:
                    for genre1 in genre_list:
                        genre1 = genre1.strip()
                        for column in genre_weights.columns:
                            if any(genre_part.strip() == genre1 for genre_part in column.split('/')):
                                genre_weight = genre_weights[column].values[0]
                                genre_weights_list.append(genre_weight)  # 각 장르 가중치를 리스트에 추가
                                break

                    # 가장 큰 두 개의 가중치를 선택하여 합산
                    if len(genre_weights_list) >= 2:
                        genre_weights_list.sort(reverse=True)
                        genre_weight_score = genre_weights_list[0] + genre_weights_list[1]
                        genre_weight_score = genre_weight_score/2  # 두 개의 가중치를 반영한 점수 계산
                    else:
                        genre_weight_score = sum(genre_weights_list)  # 장르가 두 개 미만일 경우 모든 가중치를 합산

                # 선호 장르 가중치 적용
                favored_genre_weight_score = 0
                if favored_genres:
                    for favor_genre in favored_genres:
                        if favor_genre in genre_list:
                            favored_genre_weight_score += weights['favored_genre_weight']

                # 기존 점수와 새로운 가중치를 반영한 점수 계산
                total_score = sim_score + genre_score + director_score + actor_score + genre_weight_score + favored_genre_weight_score
                # 디버깅 또는 로그 출력을 위해 idx와 관련 정보를 출력
                sim_scores.append((filtered_idx, total_score))
                # print(f'Idx: {filtered_idx}, Movie Title: {data.loc[filtered_idx, "영화명"]}, '
                #     f'Similarity: {sim_score}, Genre Score: {genre_score}, '
                #     f'Director Score: {director_score},'
                #     f'Actor Score: {actor_score},'
                #     f'Genre Weight Score: {genre_weight_score},'
                #     f'favored_genre_weight_score: {favored_genre_weight_score},'
                #     f'Total Score: {total_score}' )
            # 유사도 순으로 정렬
            sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
            print(sim_scores)
            # 상위 5개 추천
            sim_scores = sim_scores[:5]
            movie_indices = [i[0] for i in sim_scores]
            recommendations = data.iloc[movie_indices]

            # 시각화를 위한 유사도 점수 포함
            similarity_scores = [i[1] for i in sim_scores]
            return recommendations[['영화명','장르']].values.tolist(), similarity_scores

    else:
            # movie_title이 없는 경우 나머지 조건들을 사용하여 추천
        print(genre)
        sim_scores = []
        print("nnp_and_nng_review 내용:", nnp_and_nng_review)
        exclude_words = ['영화', '추천','내용','줄거리']
        words_only = [word for word, tag in nnp_and_nng_review if word not in exclude_words]
        words_text = ' '.join(words_only)
        print("words_text 내용:", words_text)
        if any(word == '내용' or word == '줄거리' for word, tag in nnp_and_nng_review):
            # user_input을 벡터화한 것과 영화를 비교하는 로직
            user_tfidf_vector = tfidf.transform([tokenize_and_lemmatize(words_text)])
            cosine_sim_user = linear_kernel(user_tfidf_vector, tfidf_matrix_synopsys)

            for idx in filtered_data.index:
                sim_score = cosine_sim_user[0, idx]

                # 최종 점수 계산 및 저장
                total_score = sim_score  # 여기에 필요한 가중치를 추가하여 계산할 수 있습니다.
                sim_scores.append((idx, total_score))

            # 유사도 점수로 정렬 후 상위 5개를 선택
            sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[:5]
            movie_indices = [i[0] for i in sim_scores]
            similarity_scores = [i[1] for i in sim_scores]

            # 영화 제목과 장르를 리스트로 반환
            recommendations = data.iloc[movie_indices][['영화명', '평점', '연도', '상영시간', '연령', '감독', '출연진', '줄거리', '장르']].values.tolist()
            return recommendations, similarity_scores
        else:
            filtered_indices = filtered_data.index
            for idx in filtered_indices:
                genre_score = 0
                year_score = 0  # 각 영화마다 year_score을 초기화

                # '최신'이란 단어가 있으면 연도에 따라 점수 부여
                if any(word == '최신' for word, tag in nnp_and_nng_review):
                    if data.loc[idx, '연도'] == 2024:
                        year_score = 2

                if genre:
                    # 입력된 장르를 쉼표로 구분하여 소문자로 변환하고 공백을 제거한 후 리스트로 만듦
                    input_genres = [g.strip().lower() for g in genre.split(',')]
                    # 영화의 장르를 소문자로 변환하고 공백을 제거한 후 리스트로 만듦
                    searched_m_genres = [g.strip().lower() for g in data.loc[idx, '장르'].split(',')]
                    # 입력된 각 장르에 대해 영화의 장르 목록과 비교하여 일치하는 경우 가중치 적용
                    genre_score = 0  # 장르 점수 초기화
                    for input_genre in input_genres:
                        if input_genre in searched_m_genres:
                            genre_score += weights['genre_weight']  # 일치하는 장르마다 가중치 추가

                director_score = weights['director_weight'] if director and director in data.loc[idx, '감독'] else 0
                actor_score = weights['actor_weight'] if actor and actor in data.loc[idx, '출연진'] else 0

                genre_list = data.loc[idx, '장르'].split(',')
                genre_weight_score = 0
                genre_weights_list = []

                if gender and age:
                    for genre1 in genre_list:
                        genre1 = genre1.strip()
                        for column in genre_weights.columns:
                            if any(genre_part.strip() == genre1 for genre_part in column.split('/')):
                                genre_weight = genre_weights[column].values[0]
                                genre_weights_list.append(genre_weight)
                                break

                    if len(genre_weights_list) >= 2:
                        genre_weights_list.sort(reverse=True)
                        genre_weight_score = genre_weights_list[0] + genre_weights_list[1]
                        genre_weight_score /= 2
                    else:
                        genre_weight_score = sum(genre_weights_list)

                favored_genre_weight_score = 0
                if favored_genres:
                    for favor_genre in favored_genres:
                        if favor_genre in genre_list:
                            favored_genre_weight_score += weights['favored_genre_weight']

                total_score = genre_score + director_score + actor_score + genre_weight_score + favored_genre_weight_score + year_score
                sim_scores.append((idx, total_score))
                # print(f'Idx: {idx}, Movie Title: {data.loc[idx, "영화명"]}, '
                #     f'Genre Score: {genre_score}, Director Score: {director_score}, '
                #     f'Actor Score: {actor_score}, Genre Weight Score: {genre_weight_score}, '
                #     f'Favored Genre Weight Score: {favored_genre_weight_score}, '
                #     f'Year Score: {year_score}, '
                #     f'Total Score: {total_score}')
            print(genre)
            sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
            print(sim_scores)
            # 상위 5개 출력
            sim_scores = sim_scores[:5]
            movie_indices = [i[0] for i in sim_scores]
            recommendations = data.iloc[movie_indices]
            similarity_scores = [i[1] for i in sim_scores]
            return recommendations[['영화명','장르']].values.tolist(), similarity_scores



In [52]:
# 사용자 입력 처리 및 영화 추천
user_input = input("영화 제목, 원하는 장르, 감독 또는 출연 배우를 입력하세요 (예: 공포, 액션, 멜로, 서부, 로버트 다우니 주니어): ")

# 전처리 및 명사 필터링
filtered_review, nnp_and_nng_review = preprocessing(user_input, komoran, remove_stopwords=False, stop_words=None)

# 일치하는 영화명 찾기
matching_movie_info = find_matching_movie(filtered_review, movie_name_genre)

# 일치하는 출연진 찾기
matching_cast = find_matching_cast(filtered_review, cast_members)

# 일치하는 장르 찾기
matching_genres = find_matching_genre(nnp_and_nng_review, genres)
print(matching_genres)
# 일치하는 감독 찾기
matching_director = find_matching_genre(filtered_review, directors)

# 결과 출력
if matching_movie_info is not None and not matching_movie_info.empty:
    movie_found = matching_movie_info['영화명'].tolist()[0]
else:
    movie_found = None

director_found = matching_director if matching_director else None
genre_found = matching_genres if matching_genres else None
person_found = matching_cast if matching_cast else None

print("고유명사(NNP)만 추출한 결과:", filtered_review)
print("고유명사(NNP)와 일반명사(NNG) 추출한 결과:", nnp_and_nng_review)
print(f"찾은 영화: {movie_found}")
print(f"찾은 감독: {director_found}")
print(f"찾은 장르: {genre_found}")
print(f"찾은 인물: {person_found}")
# 범죄도시2 같은 액션 영화 추천해줘
# 범죄도시2 같은 액션 마동석 나오는 영화 추천 해줘
# 키즈모노가타리같은 애니 영화 추천 해줘
# 최신 액션 영화 추천 해줘
# 전쟁나서 사람들이 피난가는 줄거리의 영화를 추천해줄래?
# ['영화명','장르', '평점', '연도', '상영시간', '연령', '감독', '출연', '시놉시스']
# 사용자 입력을 통한 영화명, 장르, 감독, 제작 연도, 평점 입력
movie_title = movie_found
genre = genre_found
print(genre)
director = director_found
actor = person_found
# 성별, 나이, 선호 장르 입력 처리
gender_input = input("사용자의 성별을 입력하세요 (남성, 여성) (입력하지 않으면 무시됩니다): ").strip()
age_input = input("사용자의 나이를 입력하세요 (입력하지 않으면 무시됩니다): ").strip()
favored_genres_input = input("사용자의 선호 장르를 입력하세요 (입력하지 않으면 무시됩니다): ").strip()

# 입력 값이 비어 있으면 None 또는 기본값으로 설정
gender = gender_input if gender_input else None
age = int(age_input) if age_input.isdigit() else None
favored_genres = favored_genres_input.split() if favored_genres_input else None
print(f"전달될 장르: {genre}")
recommendations, similarity_scores = get_recommendations(movie_title=movie_title, genre=genre, director=director, actor=actor, gender=gender, age=age, favored_genres=favored_genres)
print(recommendations)
print(similarity_scores)
# 범죄도시2 같은 액션 영화 추천해줘
# 범죄도시2 같은 액션 마동석 나오는 영화 추천 해줘
# 키즈모노가타리같은 애니 영화 추천 해줘
# 최신 액션 영화 추천 해줘
# 전쟁나서 사람들이 피난가는 줄거리의 영화를 추천해줄래?
# 전쟁나서 사람들이 피난가는 줄거리의 영화를 추천해줄래?
# 스포츠 코디디 영화 추천 해줘
# 코미디 스포츠 영화 추천 해줘

영화 제목, 원하는 장르, 감독 또는 출연 배우를 입력하세요 (예: 공포, 액션, 멜로, 서부, 로버트 다우니 주니어): 범죄도시2 같은 코미디 스포츠 영화 추천 해줘
코미디, 스포츠
고유명사(NNP)만 추출한 결과: [('범죄도시', 'NNP'), ('스포츠 영화', 'NNP')]
고유명사(NNP)와 일반명사(NNG) 추출한 결과: [('범죄도시', 'NNP'), ('코미디', 'NNG'), ('스포츠 영화', 'NNP'), ('추천', 'NNG')]
찾은 영화: 범죄도시4
찾은 감독: None
찾은 장르: 코미디, 스포츠
찾은 인물: None
코미디, 스포츠
사용자의 성별을 입력하세요 (남성, 여성) (입력하지 않으면 무시됩니다): 
사용자의 나이를 입력하세요 (입력하지 않으면 무시됩니다): 
사용자의 선호 장르를 입력하세요 (입력하지 않으면 무시됩니다): 
전달될 장르: 코미디, 스포츠
함수 내부에서 받은 장르: 코미디, 스포츠
코미디, 스포츠
[(903, 0.4332054357582886), (5183, 0.4300309692648129), (3150, 0.4295627749174665), (4472, 0.42385655117793486), (2018, 0.41985030300147225), (2978, 0.4170082659395495), (4755, 0.4165828131854841), (5151, 0.4165366283634858), (2798, 0.41600366324830756), (4223, 0.41535485511386117), (5250, 0.41460330035267323), (1629, 0.41284488977372513), (3749, 0.4120666402613433), (2748, 0.40599666978550625), (1312, 0.40549328407471086), (5015, 0.3871054303128062), (1042, 0.3511937199560544), (1081, 0.3410784556480482), (1645,

In [15]:


# def get_user_input():

#     user_input = input("영화 제목, 원하는 장르, 감독 또는 출연 배우를 입력하세요 (예: 공포, 액션, 멜로, 서부, 로버트 다우니 주니어): ")
#     gender_input = input("사용자의 성별을 입력하세요 (남성, 여성) (입력하지 않으면 무시됩니다): ").strip()
#     age_input = input("사용자의 나이를 입력하세요 (입력하지 않으면 무시됩니다): ").strip()
#     favored_genres_input = input("사용자의 선호 장르를 입력하세요 (입력하지 않으면 무시됩니다): ").strip()
#     return user_input, gender_input, age_input, favored_genres_input

# def process_user_input(user_input, komoran, movie_name_genre, cast_members, genres, directors):

#     # 전처리 및 명사 필터링
#     filtered_review, nnp_and_nng_review = preprocessing(user_input, komoran)

#     # 일치하는 영화명 찾기
#     matching_movie_info = find_matching_movie(filtered_review, movie_name_genre)

#     # 일치하는 출연진 찾기
#     matching_cast = find_matching_cast(filtered_review, cast_members)

#     # 일치하는 장르 찾기
#     matching_genres = find_matching_genre(nnp_and_nng_review, genres)

#     # 일치하는 감독 찾기
#     matching_director = find_matching_genre(filtered_review, directors)

#     # 결과 정리
#     movie_found = matching_movie_info['영화명'].tolist()[0] if matching_movie_info is not None and not matching_movie_info.empty else None
#     director_found = matching_director if matching_director else None
#     genre_found = matching_genres if matching_genres else None
#     person_found = matching_cast if matching_cast else None

#     return filtered_review, nnp_and_nng_review, movie_found, director_found, genre_found, person_found

# def recommend_movies(movie_title, keywords, genre, director, actor, gender, age, favored_genres, cosine_sim):
#     """
#     추천 영화를 찾고 결과를 출력합니다.
#     """
#     recommendations, similarity_scores = get_recommendations(movie_title, keywords, genre, director, actor, gender, age, favored_genres, cosine_sim)
#     print("추천된 영화 목록:")
#     for recommendation, score in zip(recommendations, similarity_scores):
#         print(f"{recommendation} - 유사도 점수: {score}")

# def main(komoran, movie_name_genre, cast_members, genres, directors, cosine_sim):
#     """
#     전체 프로세스를 실행하는 메인 함수
#     """
#     # 사용자 입력 받기
#     user_input, gender_input, age_input, favored_genres_input = get_user_input()

#     # 사용자 입력 처리 및 데이터 찾기
#     filtered_review, nnp_and_nng_review, movie_found, director_found, genre_found, person_found = process_user_input(
#         user_input, komoran, movie_name_genre, cast_members, genres, directors
#     )

#     print("고유명사(NNP)만 추출한 결과:", filtered_review)
#     print("고유명사(NNP)와 일반명사(NNG) 추출한 결과:", nnp_and_nng_review)
#     print(f"찾은 영화: {movie_found}")
#     print(f"찾은 감독: {director_found}")
#     print(f"찾은 장르: {genre_found}")
#     print(f"찾은 인물: {person_found}")

#     # 성별, 나이, 선호 장르 입력 처리
#     gender = gender_input if gender_input else None
#     age = int(age_input) if age_input.isdigit() else None
#     favored_genres = favored_genres_input.split() if favored_genres_input else None

#     # 영화 추천 - 추출된 키워드를 기반으로
#     recommend_movies(movie_found, nnp_and_nng_review, genre_found, director_found, person_found, gender, age, favored_genres, cosine_sim)

# # main 함수를 호출하여 전체 프로세스를 실행합니다.
# main(komoran, movie_name_genre, cast_members, genres, directors, cosine_sim)



In [16]:
# 범죄도시2 같은 액션 영화 추천해줘
# 범죄도시2 같은 액션 마동석 나오는 영화 추천 해줘
# 키즈모노가타리같은 애니 영화 추천 해줘
# 최신 액션 영화 추천 해줘
# 전쟁나서 사람들이 피난가는 줄거리의 영화를 추천해줄래?
# 전쟁나서 사람들이 피난가는 줄거리의 영화를 추천해줄래?

In [20]:
data[data['장르'] == '스포츠']

,영화명,평점,연도,상영시간,연령,감독,출연진,줄거리,장르
3163,글러브,8.85,2011,144분,전체 관람가,강우석,"정재영, 유선, 강신일, 조진웅, 김미경, 김혜성, 장기범, 이현우, 김동영, 김연준",소개최다 연승! 최다 탈삼진! 3년 연속 MVP! 한마디로 대한민국 프로야구 최고의...,스포츠
3394,"블루 서멀, 같은 하늘을 보고 싶어",7.38,2023,103분,전체 관람가,타치바나 마사키,"홋타 마유, 시마자키 노부나가, 에노키 준야, 코마츠 미카코, 오노 다이스케",소개“반짝거리는 사랑을 하고 싶어!”빛나는 로맨스를 꿈꾸는 신입생 ‘타마키’!하지만...,스포츠
